In [1]:
import numpy as np
import random
import math
import heapq
import networkx as nx
import matplotlib.pyplot as plt

# **Question 1**

### **Example Beam Search Visualization for your understanding**
#### **Problem**  
We have **3 machines** and **8 tasks** with durations:  
Tasks = `[4, 7, 2, 5, 8, 3, 6, 1]`  
The goal is to assign tasks to machines **minimizing makespan** (maximum load).  

---

### **Step 1: Assign Task 0 (4 minutes) to Any Machine**  

| Option | Machine 0 | Machine 1 | Machine 2 | Makespan |
|--------|------------|------------|------------|------------|
| Assign Task 0 | `4` | `0` | `0` | **4** |
| Assign Task 0 | `0` | `4` | `0` | **4** |
| Assign Task 0 | `0` | `0` | `4` | **4** |

Keep **best `beam_width` states** (ones with lowest makespan).  

---

### **Step 2: Assign Task 1 (7 minutes)**  

For each previous state, assign **Task 1 (7 min)**.  
Example if we kept **2 best states** from Step 1:  

| Option | Machine 0 | Machine 1 | Machine 2 | Makespan |
|--------|------------|------------|------------|------------|
| Assign Task 1 | `4+7=11` | `0` | `0` | **11** |
| Assign Task 1 | `4` | `7` | `0` | **7** |
| Assign Task 1 | `4` | `0` | `7` | **7** |

Keep **best `beam_width` states**.  

---

### **Step 3: Assign Task 2 (2 minutes)**  
For each best state from Step 2, assign Task 2:  

| State | Machine 0 | Machine 1 | Machine 2 | Makespan |
|-------|------------|------------|------------|------------|
| **State A** | `11` | `0` | `0` | **11** |
| **State B** | `4` | `7` | `0` | **7** |

#### **Expand: Assign Task 2 (2 min)**  

| New State | Machine 0 | Machine 1 | Machine 2 | Makespan |
|------------|------------|------------|------------|------------|
| **State A1** | `13` | `0` | `0` | **13** |
| **State A2** | `11` | `2` | `0` | **11** |
| **State A3** | `11` | `0` | `2` | **11** |
| **State B1** | `6` | `7` | `0` | **7** |
| **State B2** | `4` | `9` | `0` | **9** |
| **State B3** | `4` | `7` | `2` | **7** |

Keep **best `beam_width` states**:  
*   `State B1 → [6, 7, 0]` (**makespan 7**)  
*   `State B3 → [4, 7, 2]` (**makespan 7**)  

---

### **Continue Assigning Tasks Until All Tasks Are Assigned**  
- Repeat the same process for all tasks.  
- Expand states → Compute makespan → Keep best `beam_width` states.  
- The final state with the lowest makespan is the **optimal schedule**.  

---

### **Final Output Example**

| Machine 0 | Machine 1 | Machine 2 | Final Makespan |
|------------|------------|------------|------------|
| `12` | `15` | `14` | **15** |

 **Final Assignments:**  
`[(0,0), (1,1), (2,2), (3,0), (4,1), (5,2), (6,0), (7,1)]`  

---

**What Beam Search do?**  
*   **Beam Search efficiently assigns tasks without brute force.**  
*   **Only the best `beam_width` states are explored, making it faster.**  
*   **Minimizing makespan ensures fair task distribution.**  

 **Now, implement it. Best of Luck!**


In [21]:
def beam_search_task_scheduling(tasks, num_machines, beam_width):
    """
    Use Beam Search to assign tasks to machines while minimizing makespan.

    Return:
        best_schedule (list of tuples): Task assignments as (task_id, machine_id).
        final_machine_loads (list): Total load (time) on each machine.
    """

    # Initialize the search space (Initial State)
    initial_state = (0, [], [0] * num_machines)  # (task assignments, machine loads)
    queue = [initial_state]  # priority queue to track best states

    # TODO: Implement Beam Search Logic Here
    for task_id, task_time in enumerate(tasks):
        next_states = []

        for makespan, schedule, machine_loads in queue:
            for machine_id in range(num_machines):
                new_machine_loads = machine_loads[:]
                new_machine_loads[machine_id] += task_time
                new_makespan = max(new_machine_loads)
                new_schedule = schedule + [(task_id, machine_id)]

                next_states.append((new_makespan, new_schedule, new_machine_loads))

        # Keep only the best `beam_width` states
        queue = heapq.nsmallest(beam_width, next_states, key=lambda x: x[0])

    # The best state in the queue is our solution
    best_makespan, best_schedule, final_machine_loads = queue[0]
    return final_machine_loads, best_schedule, best_makespan
    return


In [23]:
##### TEST CASE 1 #######
tasks = [4, 7, 2, 5, 8, 3, 6, 1]
num_machines = 3
beam_width = 1

final_loads, assignments, makespan = beam_search_task_scheduling(tasks, num_machines, beam_width)

print("Final Machine Loads:", final_loads)
print("Task Assignments:", assignments)
print("Final Makespan:", makespan)

Final Machine Loads: [10, 13, 13]
Task Assignments: [(0, 0), (1, 1), (2, 0), (3, 2), (4, 2), (5, 0), (6, 1), (7, 0)]
Final Makespan: 13


In [22]:
##### TEST CASE 2 #######
tasks = [18, 5, 12, 3, 20, 7, 14, 8, 10, 6, 9, 4]
machines = 3

for beam_width in [1, 2, 3, 4]:
    print(f"\nRunning Beam Search with beam_width = {beam_width}")
    final_loads, assignments, makespan = beam_search_task_scheduling(tasks, machines, beam_width)
    print(f"Final Machine Loads: {final_loads}")
    print(f"Task Assignments: {assignments}")
    print(f"Final Makespan: {makespan}")


Running Beam Search with beam_width = 1
Final Machine Loads: [38, 41, 37]
Task Assignments: [(0, 0), (1, 1), (2, 1), (3, 2), (4, 2), (5, 1), (6, 0), (7, 1), (8, 2), (9, 0), (10, 1), (11, 2)]
Final Makespan: 41

Running Beam Search with beam_width = 2
Final Machine Loads: [39, 37, 40]
Task Assignments: [(0, 0), (1, 1), (2, 1), (3, 2), (4, 2), (5, 0), (6, 1), (7, 2), (8, 0), (9, 1), (10, 2), (11, 0)]
Final Makespan: 40

Running Beam Search with beam_width = 3
Final Machine Loads: [39, 37, 40]
Task Assignments: [(0, 0), (1, 1), (2, 1), (3, 2), (4, 2), (5, 0), (6, 1), (7, 2), (8, 0), (9, 1), (10, 2), (11, 0)]
Final Makespan: 40

Running Beam Search with beam_width = 4
Final Machine Loads: [39, 37, 40]
Task Assignments: [(0, 0), (1, 1), (2, 1), (3, 2), (4, 2), (5, 0), (6, 1), (7, 2), (8, 0), (9, 1), (10, 2), (11, 0)]
Final Makespan: 40


# **Question 2**

In [51]:
import random
import math

def simulated_annealing(initial_temp, target_temp, cooling_rates, base_energy_costs, cooling_factor, max_iterations, alpha, beta):
    """
    Simulated Annealing algorithm to optimize energy-efficient cooling.

    Parameters:
    - target_temp: The desired room temperature (°C).
    - max_iterations: Maximum number of iterations before stopping.
    - alpha: Weight for temperature difference in heuristic function.
    - beta: Weight for energy consumption in heuristic function.

    TODO:
    - Implement the main Simulated Annealing loop:
        - Take cooling rate.
        - Calculate new temperature (ensure it doesn't go below target).
        - Compute heuristic function h(T).
        - Decide whether to accept the new state based on probability.
        - Reduce temperature gradually.
    - Stop when the target temperature is reached.
    """


    # Initialize the system state
    SA_temp = 100
    decisions = 0
    time_elapsed = 0

    total_energy = 0
    current_temp = initial_temp
    cooling_rate = random.choice(cooling_rates)
    current_temp = current_temp - cooling_rate
    new_temp = current_temp
    total_energy += energy_costs[cooling_rates.index(cooling_rate)]

    for iteration in range(max_iterations):

        h_T_ = alpha * abs(current_temp - target_temp) + beta * energy_costs[cooling_rates.index(cooling_rate)]

        # TODO: Check stopping condition (if current temperature is below target)
        if new_temp <= target_temp:
            break

        # TODO: Select a random cooling rate and calculate new temperature
        new_cooling_rate = random.choice(cooling_rates)
        new_temp = new_temp - cooling_rate

        # TODO: Compute heuristic function h(T)
        h_T_new = alpha * abs(new_temp - target_temp) + beta * energy_costs[cooling_rates.index(cooling_rate)]
        energy_costs[cooling_rates.index(cooling_rate)] = energy_costs[cooling_rates.index(cooling_rate)] + random.randint(0, 40)

        # TODO: Decide whether to accept the new state based on probability
        if h_T_new < h_T_:
            current_temp = new_temp
            h_T_ = h_T_new
            cooling_rate = new_cooling_rate
            decisions += 1
        else:
            probability = math.exp((h_T_ - h_T_new) / SA_temp)
            if random.random() < probability:
                current_temp = new_temp
                h_T_ = h_T_new
                cooling_rate = new_cooling_rate
                decisions += 1

        # TODO: Gradually decrease temperature
        SA_temp *= cooling_factor

        # TODO: Print iteration details (for debugging)
        print(f"Iteration {iteration + 1}:")
        print(f"Current Temperature: {current_temp}°C")
        print(f"Cooling Rate: {cooling_rate}°C/min")
        time_elapsed +=1
        total_energy += energy_costs[cooling_rates.index(cooling_rate)]

    return decisions, time_elapsed, total_energy



In [52]:
# problem parameters
initial_temp = 30
target_temp = 22
cooling_rates = [0.5, 1.0, 1.5, 2.0]  # °C per minute
energy_costs = [100, 150, 200, 250]  # Base Watts (plus additional variation by random number addition)
cooling_factor = 0.95
max_iterations = 80
alpha = 1
beta = 0.2

decisions, time_elapsed, total_energy = simulated_annealing(initial_temp, target_temp, cooling_rates, energy_costs, cooling_factor, max_iterations, alpha, beta)

print("Cooling Decisions:", decisions)
print("Total Time Required:", time_elapsed, "minutes")
print("Total Energy Consumed:", total_energy, "Watts")

Iteration 1:
Current Temperature: 29.0°C
Cooling Rate: 0.5°C/min
Iteration 2:
Current Temperature: 28.5°C
Cooling Rate: 1.5°C/min
Iteration 3:
Current Temperature: 27.0°C
Cooling Rate: 1.0°C/min
Iteration 4:
Current Temperature: 26.0°C
Cooling Rate: 0.5°C/min
Iteration 5:
Current Temperature: 25.5°C
Cooling Rate: 0.5°C/min
Iteration 6:
Current Temperature: 25.0°C
Cooling Rate: 0.5°C/min
Iteration 7:
Current Temperature: 24.5°C
Cooling Rate: 2.0°C/min
Iteration 8:
Current Temperature: 22.5°C
Cooling Rate: 1.0°C/min
Iteration 9:
Current Temperature: 21.5°C
Cooling Rate: 0.5°C/min
Cooling Decisions: 9
Total Time Required: 9 minutes
Total Energy Consumed: 1681 Watts
